In [143]:
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
import tensorflow as tf
from keras.datasets import mnist
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np

In [51]:
img_rows = 28 
img_cols = 28
channels = 1

In [52]:
optimizer = Adam(0.0002, 0.5)

In [114]:
def build_generator():

    noise_shape = (100,)
    x_noise = Input(shape=noise_shape)

    x = Dense(128 * 7 * 7, activation="relu")(x_noise)
    x = Reshape((7, 7, 128))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D()(x)
    x = Conv2D(128, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D()(x)
    x = Conv2D(64, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(1, kernel_size=3, padding="same")(x)
    gen_out = Activation("tanh")(x)

    return Model(x_noise, gen_out)

In [115]:
# Build and compile the generator
generator = build_generator()
generator.summary()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# The generator takes noise as input and generated imgs
z = Input(shape=(100,))
img = generator(z)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 6272)              633472    
_________________________________________________________________
reshape_10 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_19 (UpSampling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_28 (Activation)   (None, 14, 14, 128)       0         
__________

In [116]:
def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

In [191]:
def build_discriminator():

        img_shape = (img_rows, img_cols, channels)
        x_img = Input(shape=img_shape)

        x = Conv2D(32, kernel_size=3, strides=2, padding="same")(x_img)
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
        x = ZeroPadding2D(padding=((0,1),(0,1)))(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.25)(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Conv2D(128, kernel_size=3, strides=2, padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.25)(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Conv2D(256, kernel_size=3, strides=1, padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.25)(x)

        x = Flatten()(x)
        pred = Dense(1, activation='sigmoid')(x)

        return Model(x_img, pred)

In [192]:
# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.summary()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_60 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

In [193]:
# For the combined model we will only train the generator
discriminator.trainable = True

# The valid takes generated images as input and determines validity
valid = discriminator(img)

In [194]:
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity 
combined = Model(z, valid)
combined.summary()
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 100)               0         
_________________________________________________________________
model_22 (Model)             (None, 28, 28, 1)         856705    
_________________________________________________________________
model_40 (Model)             (None, 1)                 392705    
Total params: 1,249,410
Trainable params: 1,248,386
Non-trainable params: 1,024
_________________________________________________________________


In [195]:
def save_imgs(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        #fig.suptitle("DCGAN: Generated digits", fontsize=12)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [196]:
def train(epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                save_imgs(epoch)

In [197]:
train(epochs=10000, batch_size=32, save_interval=50)

0 [D loss: 7.694340, acc.: 31.25%] [G loss: 8.117529]
1 [D loss: 4.729609, acc.: 50.00%] [G loss: 5.570066]
2 [D loss: 3.368826, acc.: 50.00%] [G loss: 3.000200]
3 [D loss: 1.862221, acc.: 50.00%] [G loss: 2.183062]
4 [D loss: 3.038689, acc.: 50.00%] [G loss: 1.620419]
5 [D loss: 2.237319, acc.: 50.00%] [G loss: 2.050678]
6 [D loss: 1.889169, acc.: 50.00%] [G loss: 1.808847]
7 [D loss: 1.811247, acc.: 50.00%] [G loss: 1.011471]
8 [D loss: 1.551792, acc.: 50.00%] [G loss: 1.040543]
9 [D loss: 1.189596, acc.: 50.00%] [G loss: 1.164231]
10 [D loss: 1.148851, acc.: 50.00%] [G loss: 0.993936]
11 [D loss: 1.207152, acc.: 56.25%] [G loss: 1.054651]
12 [D loss: 1.220714, acc.: 50.00%] [G loss: 0.913172]
13 [D loss: 1.476507, acc.: 50.00%] [G loss: 1.060679]
14 [D loss: 1.076116, acc.: 50.00%] [G loss: 0.797255]
15 [D loss: 1.280947, acc.: 50.00%] [G loss: 0.804518]
16 [D loss: 1.335055, acc.: 50.00%] [G loss: 1.012975]
17 [D loss: 1.446313, acc.: 50.00%] [G loss: 1.104838]
18 [D loss: 1.014930

149 [D loss: 0.946824, acc.: 50.00%] [G loss: 0.412481]
150 [D loss: 1.109449, acc.: 50.00%] [G loss: 0.364476]
151 [D loss: 1.156419, acc.: 50.00%] [G loss: 0.586406]
152 [D loss: 0.995706, acc.: 50.00%] [G loss: 0.591860]
153 [D loss: 1.070668, acc.: 50.00%] [G loss: 0.393047]
154 [D loss: 1.005510, acc.: 53.12%] [G loss: 0.426206]
155 [D loss: 1.152684, acc.: 50.00%] [G loss: 0.456032]
156 [D loss: 1.093489, acc.: 53.12%] [G loss: 0.469981]
157 [D loss: 1.223736, acc.: 50.00%] [G loss: 0.419004]
158 [D loss: 1.197457, acc.: 56.25%] [G loss: 0.470250]
159 [D loss: 1.250559, acc.: 53.12%] [G loss: 0.412237]
160 [D loss: 1.217535, acc.: 53.12%] [G loss: 0.339230]
161 [D loss: 1.008535, acc.: 46.88%] [G loss: 0.402760]
162 [D loss: 0.972969, acc.: 50.00%] [G loss: 0.537748]
163 [D loss: 1.121261, acc.: 50.00%] [G loss: 0.574815]
164 [D loss: 0.998722, acc.: 53.12%] [G loss: 0.402108]
165 [D loss: 1.110204, acc.: 50.00%] [G loss: 0.426496]
166 [D loss: 1.444441, acc.: 46.88%] [G loss: 0.

296 [D loss: 1.077024, acc.: 53.12%] [G loss: 0.372721]
297 [D loss: 1.070418, acc.: 53.12%] [G loss: 0.382269]
298 [D loss: 0.993551, acc.: 56.25%] [G loss: 0.324603]
299 [D loss: 1.204103, acc.: 50.00%] [G loss: 0.411364]
300 [D loss: 1.030542, acc.: 53.12%] [G loss: 0.383061]
301 [D loss: 1.132265, acc.: 50.00%] [G loss: 0.437012]
302 [D loss: 1.191472, acc.: 50.00%] [G loss: 0.410336]
303 [D loss: 0.919482, acc.: 53.12%] [G loss: 0.339279]
304 [D loss: 0.947857, acc.: 50.00%] [G loss: 0.378233]
305 [D loss: 1.189948, acc.: 50.00%] [G loss: 0.458906]
306 [D loss: 0.920339, acc.: 50.00%] [G loss: 0.469675]
307 [D loss: 1.207223, acc.: 50.00%] [G loss: 0.391635]
308 [D loss: 1.139946, acc.: 50.00%] [G loss: 0.293650]
309 [D loss: 1.092990, acc.: 50.00%] [G loss: 0.375211]
310 [D loss: 1.074825, acc.: 53.12%] [G loss: 0.374839]
311 [D loss: 1.126905, acc.: 46.88%] [G loss: 0.365944]
312 [D loss: 0.952161, acc.: 56.25%] [G loss: 0.337252]
313 [D loss: 1.119980, acc.: 53.12%] [G loss: 0.

443 [D loss: 1.226632, acc.: 50.00%] [G loss: 0.284177]
444 [D loss: 0.996984, acc.: 53.12%] [G loss: 0.389017]
445 [D loss: 0.992113, acc.: 53.12%] [G loss: 0.358270]
446 [D loss: 1.162938, acc.: 50.00%] [G loss: 0.374508]
447 [D loss: 0.937680, acc.: 50.00%] [G loss: 0.330163]
448 [D loss: 1.122892, acc.: 50.00%] [G loss: 0.273946]
449 [D loss: 1.099565, acc.: 53.12%] [G loss: 0.419173]
450 [D loss: 1.012966, acc.: 50.00%] [G loss: 0.318187]
451 [D loss: 1.217529, acc.: 53.12%] [G loss: 0.346340]
452 [D loss: 1.165790, acc.: 43.75%] [G loss: 0.337562]
453 [D loss: 1.079491, acc.: 50.00%] [G loss: 0.395182]
454 [D loss: 1.104038, acc.: 46.88%] [G loss: 0.382813]
455 [D loss: 1.041841, acc.: 50.00%] [G loss: 0.316037]
456 [D loss: 1.219268, acc.: 53.12%] [G loss: 0.363956]
457 [D loss: 1.201540, acc.: 50.00%] [G loss: 0.327202]
458 [D loss: 0.922358, acc.: 53.12%] [G loss: 0.336088]
459 [D loss: 1.087470, acc.: 50.00%] [G loss: 0.362697]
460 [D loss: 1.124130, acc.: 50.00%] [G loss: 0.

590 [D loss: 1.094283, acc.: 50.00%] [G loss: 0.313749]
591 [D loss: 1.102084, acc.: 46.88%] [G loss: 0.338783]
592 [D loss: 1.198007, acc.: 53.12%] [G loss: 0.343919]
593 [D loss: 1.005939, acc.: 53.12%] [G loss: 0.358873]
594 [D loss: 1.037342, acc.: 50.00%] [G loss: 0.284801]
595 [D loss: 0.922290, acc.: 50.00%] [G loss: 0.306990]
596 [D loss: 1.125641, acc.: 50.00%] [G loss: 0.294190]
597 [D loss: 1.054444, acc.: 50.00%] [G loss: 0.417064]
598 [D loss: 1.109362, acc.: 50.00%] [G loss: 0.377485]
599 [D loss: 0.906226, acc.: 50.00%] [G loss: 0.301840]
600 [D loss: 0.949110, acc.: 46.88%] [G loss: 0.278060]
601 [D loss: 1.245141, acc.: 50.00%] [G loss: 0.268666]
602 [D loss: 0.856658, acc.: 50.00%] [G loss: 0.290052]
603 [D loss: 1.014336, acc.: 50.00%] [G loss: 0.445822]
604 [D loss: 0.957017, acc.: 50.00%] [G loss: 0.382353]
605 [D loss: 1.106879, acc.: 53.12%] [G loss: 0.343218]
606 [D loss: 1.172146, acc.: 56.25%] [G loss: 0.293057]
607 [D loss: 1.065817, acc.: 53.12%] [G loss: 0.

...

9404 [D loss: 0.997348, acc.: 50.00%] [G loss: 0.331410]
9405 [D loss: 1.217321, acc.: 50.00%] [G loss: 0.458316]
9406 [D loss: 1.239807, acc.: 46.88%] [G loss: 0.321292]
9407 [D loss: 1.119171, acc.: 53.12%] [G loss: 0.344531]
9408 [D loss: 1.111462, acc.: 53.12%] [G loss: 0.343195]
9409 [D loss: 1.077772, acc.: 53.12%] [G loss: 0.320325]
9410 [D loss: 1.041272, acc.: 50.00%] [G loss: 0.267980]
9411 [D loss: 1.021085, acc.: 46.88%] [G loss: 0.282766]
9412 [D loss: 1.010170, acc.: 46.88%] [G loss: 0.401843]
9413 [D loss: 0.963243, acc.: 53.12%] [G loss: 0.291463]
9414 [D loss: 1.159813, acc.: 50.00%] [G loss: 0.321435]
9415 [D loss: 1.148924, acc.: 46.88%] [G loss: 0.419799]
9416 [D loss: 1.167222, acc.: 53.12%] [G loss: 0.464094]
9417 [D loss: 1.099100, acc.: 53.12%] [G loss: 0.268149]
9418 [D loss: 1.203899, acc.: 50.00%] [G loss: 0.325979]
9419 [D loss: 1.306449, acc.: 46.88%] [G loss: 0.346664]
9420 [D loss: 1.116991, acc.: 53.12%] [G loss: 0.314617]
9421 [D loss: 0.987290, acc.: 5

9550 [D loss: 1.539618, acc.: 53.12%] [G loss: 0.302406]
9551 [D loss: 1.089536, acc.: 46.88%] [G loss: 0.373024]
9552 [D loss: 1.212070, acc.: 53.12%] [G loss: 0.485504]
9553 [D loss: 1.133834, acc.: 50.00%] [G loss: 0.292452]
9554 [D loss: 1.196051, acc.: 50.00%] [G loss: 0.482613]
9555 [D loss: 1.009711, acc.: 53.12%] [G loss: 0.370519]
9556 [D loss: 1.201658, acc.: 50.00%] [G loss: 0.313679]
9557 [D loss: 1.105987, acc.: 50.00%] [G loss: 0.416680]
9558 [D loss: 1.023929, acc.: 53.12%] [G loss: 0.248931]
9559 [D loss: 1.268390, acc.: 50.00%] [G loss: 0.386209]
9560 [D loss: 1.098547, acc.: 56.25%] [G loss: 0.290953]
9561 [D loss: 1.071484, acc.: 50.00%] [G loss: 0.296195]
9562 [D loss: 0.948642, acc.: 53.12%] [G loss: 0.288534]
9563 [D loss: 1.141887, acc.: 46.88%] [G loss: 0.297487]
9564 [D loss: 1.183140, acc.: 50.00%] [G loss: 0.291713]
9565 [D loss: 1.013172, acc.: 53.12%] [G loss: 0.364799]
9566 [D loss: 0.857588, acc.: 46.88%] [G loss: 0.458335]
9567 [D loss: 1.051450, acc.: 5

9696 [D loss: 1.257676, acc.: 46.88%] [G loss: 0.321330]
9697 [D loss: 1.261003, acc.: 46.88%] [G loss: 0.283126]
9698 [D loss: 1.119427, acc.: 53.12%] [G loss: 0.246912]
9699 [D loss: 0.929727, acc.: 53.12%] [G loss: 0.336920]
9700 [D loss: 0.985669, acc.: 59.38%] [G loss: 0.503373]
9701 [D loss: 0.935605, acc.: 56.25%] [G loss: 0.381445]
9702 [D loss: 1.328749, acc.: 43.75%] [G loss: 0.361500]
9703 [D loss: 1.012936, acc.: 50.00%] [G loss: 0.295465]
9704 [D loss: 1.039241, acc.: 50.00%] [G loss: 0.328941]
9705 [D loss: 1.357248, acc.: 50.00%] [G loss: 0.319255]
9706 [D loss: 0.977103, acc.: 50.00%] [G loss: 0.396145]
9707 [D loss: 1.162506, acc.: 50.00%] [G loss: 0.323369]
9708 [D loss: 1.047971, acc.: 43.75%] [G loss: 0.343056]
9709 [D loss: 1.257356, acc.: 46.88%] [G loss: 0.406740]
9710 [D loss: 1.001853, acc.: 53.12%] [G loss: 0.277311]
9711 [D loss: 1.231944, acc.: 50.00%] [G loss: 0.351937]
9712 [D loss: 1.123993, acc.: 50.00%] [G loss: 0.391011]
9713 [D loss: 1.091007, acc.: 5

9840 [D loss: 0.876876, acc.: 59.38%] [G loss: 0.255797]
9841 [D loss: 1.232512, acc.: 50.00%] [G loss: 0.408578]
9842 [D loss: 1.195589, acc.: 40.62%] [G loss: 0.405077]
9843 [D loss: 1.292296, acc.: 50.00%] [G loss: 0.359242]
9844 [D loss: 1.129813, acc.: 50.00%] [G loss: 0.487647]
9845 [D loss: 1.227070, acc.: 50.00%] [G loss: 0.388779]
9846 [D loss: 1.004300, acc.: 53.12%] [G loss: 0.247738]
9847 [D loss: 0.946702, acc.: 59.38%] [G loss: 0.386897]
9848 [D loss: 1.186070, acc.: 50.00%] [G loss: 0.302411]
9849 [D loss: 1.135235, acc.: 50.00%] [G loss: 0.373759]
9850 [D loss: 1.275187, acc.: 43.75%] [G loss: 0.288416]
9851 [D loss: 1.005791, acc.: 50.00%] [G loss: 0.289522]
9852 [D loss: 1.141688, acc.: 50.00%] [G loss: 0.371971]
9853 [D loss: 1.049499, acc.: 53.12%] [G loss: 0.410777]
9854 [D loss: 1.045873, acc.: 53.12%] [G loss: 0.326064]
9855 [D loss: 1.288734, acc.: 50.00%] [G loss: 0.290585]
9856 [D loss: 1.192477, acc.: 46.88%] [G loss: 0.351047]
9857 [D loss: 1.180763, acc.: 5

9985 [D loss: 0.811879, acc.: 56.25%] [G loss: 0.341665]
9986 [D loss: 1.122228, acc.: 46.88%] [G loss: 0.357141]
9987 [D loss: 1.287393, acc.: 50.00%] [G loss: 0.283749]
9988 [D loss: 1.108811, acc.: 50.00%] [G loss: 0.275887]
9989 [D loss: 1.149905, acc.: 46.88%] [G loss: 0.274200]
9990 [D loss: 1.012973, acc.: 53.12%] [G loss: 0.460906]
9991 [D loss: 1.046993, acc.: 46.88%] [G loss: 0.268036]
9992 [D loss: 1.151550, acc.: 43.75%] [G loss: 0.408308]
9993 [D loss: 1.336072, acc.: 40.62%] [G loss: 0.300580]
9994 [D loss: 1.013838, acc.: 50.00%] [G loss: 0.382968]
9995 [D loss: 1.134777, acc.: 53.12%] [G loss: 0.313790]
9996 [D loss: 0.935274, acc.: 53.12%] [G loss: 0.308233]
9997 [D loss: 1.199873, acc.: 50.00%] [G loss: 0.342883]
9998 [D loss: 1.248998, acc.: 50.00%] [G loss: 0.337035]
9999 [D loss: 1.215714, acc.: 50.00%] [G loss: 0.285188]
